In [ ]:
import pandas as pd
import numpy as np



In [ ]:
data = [[1, 'Joe', 70000, 1], [2, 'Jim', 90000, 1], [3, 'Henry', 80000, 2], [4, 'Sam', 60000, 2], [5, 'Max', 90000, 1]]
employee = pd.DataFrame(data, columns=['id', 'name', 'salary', 'departmentId']).astype({'id':'Int64', 'name':'object', 'salary':'Int64', 'departmentId':'Int64'})
data = [[1, 'IT'], [2, 'Sales']]
department = pd.DataFrame(data, columns=['id', 'name']).astype({'id':'Int64', 'name':'object'})

In [ ]:
employee.head()

In [ ]:
department.head()

In [ ]:
def department_highest_salary(employee: pd.DataFrame, department: pd.DataFrame) -> pd.DataFrame:
    
    merged = employee.merge(department, left_on='departmentId', right_on='id')
    high_salaries = merged.groupby('name_y').max()[['salary', 'departmentId']]
    
    final = merged.merge(high_salaries)[['name_y', 'name_x', 'salary']]
    final = final.rename(columns={'name_y':'Department', 'name_x':'Employee','salary':'Salary'})
    return final
    
    
    

print(department_highest_salary(employee, department))
    

In [ ]:
data = [[1, 'john@example.com'], [2, 'bob@example.com'], [3, 'john@example.com']]
person = pd.DataFrame(data, columns=['id', 'email']).astype({'id':'int64', 'email':'object'})

In [ ]:
person

In [ ]:
person[~person['email'].duplicated()]

In [12]:
import pandas as pd
data = [[1, 3.5], [2, 3.65], [3, 4.0], [4, 3.85], [5, 4.0], [6, 3.65]]
scores = pd.DataFrame(data, columns=['id', 'score']).astype({'id':'Int64', 'score':'Float64'})

In [13]:
scores

,id,score
0,1,3.5
1,2,3.65
2,3,4.0
3,4,3.85
4,5,4.0
5,6,3.65


In [26]:
scores['rank'] = scores['score'].rank(ascending=False, method='dense').astype(int)

In [28]:
scores[['score', 'rank']]

,score,rank
0,3.5,4
1,3.65,3
2,4.0,1
3,3.85,2
4,4.0,1
5,3.65,3


In [1]:
config = {}
config["image_size"] = 256
config["num_layers"] = 12
config["hidden_dim"] = 768
config["mlp_dim"] = 3072
config["num_heads"] = 12
config["dropout_rate"] = 0.1
config["num_patches"] = 256
config["patch_size"] = 16
config["num_channels"] = 3

In [2]:
config

{'image_size': 256,
 'num_layers': 12,
 'hidden_dim': 768,
 'mlp_dim': 3072,
 'num_heads': 12,
 'dropout_rate': 0.1,
 'num_patches': 256,
 'patch_size': 16,
 'num_channels': 3}

In [3]:

import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model

def mlp(x, cf):
    x = L.Dense(cf["mlp_dim"], activation="gelu")(x)
    x = L.Dropout(cf["dropout_rate"])(x)
    x = L.Dense(cf["hidden_dim"])(x)
    x = L.Dropout(cf["dropout_rate"])(x)
    return x

def transformer_encoder(x, cf):
    skip_1 = x
    x = L.LayerNormalization()(x)
    x = L.MultiHeadAttention(
        num_heads=cf["num_heads"], key_dim=cf["hidden_dim"]
    )(x, x)
    x = L.Add()([x, skip_1])

    skip_2 = x
    x = L.LayerNormalization()(x)
    x = mlp(x, cf)
    x = L.Add()([x, skip_2])

    return x

def conv_block(x, num_filters, kernel_size=3):
    x = L.Conv2D(num_filters, kernel_size=kernel_size, padding="same")(x)
    x = L.BatchNormalization()(x)
    x = L.ReLU()(x)
    return x

def deconv_block(x, num_filters):
    x = L.Conv2DTranspose(num_filters, kernel_size=2, padding="same", strides=2)(x)
    return x

def build_unetr_2d(cf):
    """ Inputs """
    input_shape = (cf["num_patches"], cf["patch_size"]*cf["patch_size"]*cf["num_channels"])
    inputs = L.Input(input_shape) ## (None, 256, 768)

    """ Patch + Position Embeddings """
    patch_embed = L.Dense(cf["hidden_dim"])(inputs) ## (None, 256, 768)

    positions = tf.range(start=0, limit=cf["num_patches"], delta=1) ## (256,)
    pos_embed = L.Embedding(input_dim=cf["num_patches"], output_dim=cf["hidden_dim"])(positions) ## (256, 768)
    x = patch_embed + pos_embed ## (None, 256, 768)

    """ Transformer Encoder """
    skip_connection_index = [3, 6, 9, 12]
    skip_connections = []

    for i in range(1, cf["num_layers"]+1, 1):
        x = transformer_encoder(x, cf)

        if i in skip_connection_index:
            skip_connections.append(x)

    """ CNN Decoder """
    z3, z6, z9, z12 = skip_connections

    ## Reshaping
    z0 = L.Reshape((cf["image_size"], cf["image_size"], cf["num_channels"]))(inputs)
    z3 = L.Reshape((cf["patch_size"], cf["patch_size"], cf["hidden_dim"]))(z3)
    z6 = L.Reshape((cf["patch_size"], cf["patch_size"], cf["hidden_dim"]))(z6)
    z9 = L.Reshape((cf["patch_size"], cf["patch_size"], cf["hidden_dim"]))(z9)
    z12 = L.Reshape((cf["patch_size"], cf["patch_size"], cf["hidden_dim"]))(z12)

    ## Decoder 1
    x = deconv_block(z12, 512)

    s = deconv_block(z9, 512)
    s = conv_block(s, 512)
    x = L.Concatenate()([x, s])

    x = conv_block(x, 512)
    x = conv_block(x, 512)

    ## Decoder 2
    x = deconv_block(x, 256)

    s = deconv_block(z6, 256)
    s = conv_block(s, 256)
    s = deconv_block(s, 256)
    s = conv_block(s, 256)

    x = L.Concatenate()([x, s])
    x = conv_block(x, 256)
    x = conv_block(x, 256)

    ## Decoder 3
    x = deconv_block(x, 128)

    s = deconv_block(z3, 128)
    s = conv_block(s, 128)
    s = deconv_block(s, 128)
    s = conv_block(s, 128)
    s = deconv_block(s, 128)
    s = conv_block(s, 128)

    x = L.Concatenate()([x, s])
    x = conv_block(x, 128)
    x = conv_block(x, 128)

    ## Decoder 4
    x = deconv_block(x, 64)

    s = conv_block(z0, 64)
    s = conv_block(s, 64)

    x = L.Concatenate()([x, s])
    x = conv_block(x, 64)
    x = conv_block(x, 64)

    """ Output """
    outputs = L.Conv2D(1, kernel_size=1, padding="same", activation="sigmoid")(x)

    return Model(inputs, outputs, name="UNETR_2D")
    



In [4]:
model = build_unetr_2d(config)
model.summary()

2023-10-02 16:31:36.025359: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Max
2023-10-02 16:31:36.025377: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2023-10-02 16:31:36.025381: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2023-10-02 16:31:36.025412: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-02 16:31:36.025428: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "UNETR_2D"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 768)]           0         []                            
                                                                                                  
 dense (Dense)               (None, 256, 768)             590592    ['input_1[0][0]']             
                                                                                                  
 tf.__operators__.add (TFOp  (None, 256, 768)             0         ['dense[0][0]']               
 Lambda)                                                                                          
                                                                                                  
 layer_normalization (Layer  (None, 256, 768)             1536      ['tf.__operators__.add[

 add_5 (Add)                 (None, 256, 768)             0         ['dropout_5[0][0]',           
                                                                     'add_4[0][0]']               
                                                                                                  
 layer_normalization_6 (Lay  (None, 256, 768)             1536      ['add_5[0][0]']               
 erNormalization)                                                                                 
                                                                                                  
 multi_head_attention_3 (Mu  (None, 256, 768)             2833996   ['layer_normalization_6[0][0]'
 ltiHeadAttention)                                        8         , 'layer_normalization_6[0][0]
                                                                    ']                            
                                                                                                  
 add_6 (Ad

 yerNormalization)                                                                                
                                                                                                  
 multi_head_attention_6 (Mu  (None, 256, 768)             2833996   ['layer_normalization_12[0][0]
 ltiHeadAttention)                                        8         ',                            
                                                                     'layer_normalization_12[0][0]
                                                                    ']                            
                                                                                                  
 add_12 (Add)                (None, 256, 768)             0         ['multi_head_attention_6[0][0]
                                                                    ',                            
                                                                     'add_11[0][0]']              
          

 multi_head_attention_9 (Mu  (None, 256, 768)             2833996   ['layer_normalization_18[0][0]
 ltiHeadAttention)                                        8         ',                            
                                                                     'layer_normalization_18[0][0]
                                                                    ']                            
                                                                                                  
 add_18 (Add)                (None, 256, 768)             0         ['multi_head_attention_9[0][0]
                                                                    ',                            
                                                                     'add_17[0][0]']              
                                                                                                  
 layer_normalization_19 (La  (None, 256, 768)             1536      ['add_18[0][0]']              
 yerNormal

 conv2d (Conv2D)             (None, 32, 32, 512)          2359808   ['conv2d_transpose_1[0][0]']  
                                                                                                  
 reshape_4 (Reshape)         (None, 16, 16, 768)          0         ['add_23[0][0]']              
                                                                                                  
 batch_normalization (Batch  (None, 32, 32, 512)          2048      ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 conv2d_transpose (Conv2DTr  (None, 32, 32, 512)          1573376   ['reshape_4[0][0]']           
 anspose)                                                                                         
                                                                                                  
 re_lu (Re

                                                                                                  
 batch_normalization_8 (Bat  (None, 64, 64, 128)          512       ['conv2d_8[0][0]']            
 chNormalization)                                                                                 
                                                                                                  
 re_lu_5 (ReLU)              (None, 64, 64, 256)          0         ['batch_normalization_5[0][0]'
                                                                    ]                             
                                                                                                  
 re_lu_8 (ReLU)              (None, 64, 64, 128)          0         ['batch_normalization_8[0][0]'
                                                                    ]                             
                                                                                                  
 conv2d_6 

 conv2d_15 (Conv2D)          (None, 256, 256, 64)         36928     ['re_lu_14[0][0]']            
                                                                                                  
 batch_normalization_15 (Ba  (None, 256, 256, 64)         256       ['conv2d_15[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 re_lu_15 (ReLU)             (None, 256, 256, 64)         0         ['batch_normalization_15[0][0]
                                                                    ']                            
                                                                                                  
 conv2d_16 (Conv2D)          (None, 256, 256, 1)          65        ['re_lu_15[0][0]']            
                                                                                                  
Total para

In [8]:
# from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import plot_model

In [ ]:
plot_model(model, to_file='model.png', show_shapes=True)